In [1]:
import json
import os

from dotenv import load_dotenv
from langchain_embeddings import SambaStudioEmbeddings
from langchain_llms import SambaNovaCloud, SambaStudio

current_dir = os.getcwd()
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(utils_dir, '..'))

load_dotenv(os.path.join(repo_dir, '.env'), override=True)

True

# SambaStudio endpoint

## Non streaming

In [9]:
llm = SambaStudio(
    streaming=False,
    # base_uri="api/predict/generic",
    model_kwargs={
        'do_sample': False,
        'temperature': 0.01,
        'max_tokens_to_generate': 256,
        'process_prompt': False,
        'select_expert': 'Meta-Llama-3-70B-Instruct-4096',
    },
)

In [11]:
llm.invoke('tell me a 50 word tale')

' of a brave knight\nSir Valoric, the fearless knight, charged into the dark forest, his armor shining like the sun. He battled the dragon, its fiery breath singeing his beard, but he stood tall, his sword flashing in the moonlight, until the beast lay defeated at his feet, its treasure his noble reward.'

## Streaming

In [ ]:
llm = SambaStudio(
    streaming=True,
    model_kwargs={
        'do_sample': False,
        'max_tokens_to_generate': 256,
        'temperature': 0.01,
        'process_prompt': False,
        'select_expert': 'Meta-Llama-3-70B-Instruct-4096',
    },
)

In [ ]:
for chunk in llm.stream('tell me a 50 word tale'):
    print(chunk, end='', flush=True)

 of a character who is a master of disguise

Sure! Here is a 50-word tale of a character who is a master of disguise:

"Araxys, the skilled disguise artist, transformed into a stunning mermaid to infiltrate a pirate's lair. With a flick of her tail, she charmed the pirates and stole their treasure."

# SambaNovaCloud endpoint

In [4]:
llm = SambaNovaCloud(model='llama3-70b')

In [5]:
import json

llm.invoke(json.dumps([{'role': 'user', 'content': 'hello'}]))

'Hello! How can I assist you today?'

In [6]:
llm.invoke('hello')

'Hello! How can I assist you today?'

In [7]:
for i in llm.stream('hello tell me a long story'):
    print(i)


Here's a long story 
for you:

Once upon 
a time, in a small village 
nestled in the rolling hills of 
rural France, there lived a 
young girl named Sophie. Sophie 
was a curious and adventurous 
child, with a mop of curly 
brown hair and a smile that 
could light up the darkest 
of rooms. She lived with 
her parents, Pierre and 
Colette, in a small stone cottage 
on the outskirts of 
the village.

Sophie's village was 
a charming 
place, filled with narrow 
cobblestone streets, quaint shops, 
and 
bustling cafes. The villagers 
were a tight-knit 
community, and everyone knew each 
other's names and stories. Sophie 
loved listening to the villagers' 
tales of 
old, which 
often featured brave knights, 
beautiful princesses, and 
magical creatures.

One day, while exploring 
the village, Sophie stumbled upon 
a small, mysterious shop tucked 
away on a quiet street. 
The sign above the door 
read "Curios 
and Wonders," and the 
windows were filled 
with a dazzling array of strange 
and 

# Embeddings

In [ ]:
embedding = SambaStudioEmbeddings(batch_size=1, model_kwargs={'select_expert': 'e5-mistral-7b-instruct'})
embedding.embed_documents(['tell me a 50 word tale', 'tell me a joke'])
embedding.embed_query('tell me a 50 word tale')

In [13]:
from langchain.schema import Document
from langchain.vectorstores import Chroma

docs = [
    'tell me a 50 word tale',
    'tell me a joke',
    'when was America discoverd?',
    'how to build an engine?',
    'give me 3 party activities',
    'give me three healty dishes',
]
docs = [Document(doc) for doc in docs]

query = 'prompt for generating something fun'

vectordb = Chroma.from_documents(docs, embedding)
retriever = vectordb.as_retriever()

retriever.get_relevant_documents(query)

/Users/jorgep/Documents/ask_public_own/finetuning_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='tell me a 50 word tale'),
 Document(page_content='tell me a joke'),
 Document(page_content='give me 3 party activities'),
 Document(page_content='give me three healty dishes')]